## Installing Dependencies

In [1]:
pip install pandas scikit-learn skl2onnx onnxruntime joblib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.3/300.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.4
    Uninstalling protobuf-5.29.4:
      Successfully uninstalled protobuf-5.29.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.2 which is incompatible.
te

## Importing Dependencies

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

## Load data and Pre-process

In [2]:
df = pd.read_csv("indian_insurance_dataset.csv")
X = pd.get_dummies(df.drop("charges", axis=1),
                   columns=["sex","smoker","region"])
y = df["charges"]

## Split features/target

In [3]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

## Train plain RandomForest

In [4]:
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

## Save sklearn model for backup

In [5]:
joblib.dump(model, "insurance_model.pkl")
print("✅ Saved insurance_model.pkl")

✅ Saved insurance_model.pkl


##  Convert to ONNX

In [6]:
n_features = X_train.shape[1]
initial_type = [("float_input", FloatTensorType([None, n_features]))]

onnx_model = convert_sklearn(
    model,
    initial_types=initial_type,
    target_opset=12
)

with open("insurance_model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())
print(f"✅ Exported insurance_model.onnx ({n_features} float features)")

✅ Exported insurance_model.onnx (12 float features)


## Make predictions on the test set

In [7]:
y_pred = model.predict(X_test)

## Calculate performance metrics

In [8]:
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2  = r2_score(y_test, y_pred)

##  Print performance metrics

In [9]:
print("🔹 Performance on Test Set 🔹")
print(f"MAE:  {mae:.2f}")
print(f"MSE:  {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²:   {r2:.2f}")

🔹 Performance on Test Set 🔹
MAE:  97.58
MSE:  20894.48
RMSE: 144.55
R²:   1.00
